# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - NxNxN Cube Monte Carlo</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Using repo from dwalton76: https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master

I want to explore solving the cube after manipulating the initial state of N-labled soltuions. Modifying the initial 
state may generate a path to the solution

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
import zipfile
import sqlite3
import random
import pandas as pd
import numpy as np

import os
import itertools

os.chdir('rubiks-cube-NxNxN-solver')
# Print the current working directory
print("Current Working Directory: ", os.getcwd())

database_file = '../../solutions.db'
solution_method = "dwalton76 NxNxN algo"

Current Working Directory:  /Users/seanbearden/PycharmProjects/Kaggle/src/santa-2023/solver_hack/rubiks-cube-NxNxN-solver


<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [2]:
with zipfile.ZipFile('../../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col='id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div>
 

In [3]:
# U = ['U', 'F', 'R', 'B', 'L', 'D']

def state2ubl(state):
    
    U_dict = {
        'A': 'U',
        'C': 'R',
        'B': 'F',
        'F': 'D',
        'E': 'L',
        'D': 'B',
        
    }
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]


In [4]:
def cyclic_letters(start='A'):
    count = 6
    # Define the alphabet letters from A to F
    letters = ['A', 'B', 'C', 'D', 'E', 'F']
    # letters = ['D', 'E', 'A', 'C', 'F', 'B']
    # letters = ['D', 'A', 'C', 'F', 'E', 'B']
    # letters = ['F', 'D', 'C', 'B', 'E', 'A']
    
    # Find the index of the starting letter
    start_index = letters.index(start)
    
    # Reorder the list so that it starts with the desired letter
    ordered_letters = letters[start_index:] + letters[:start_index]
    
    # Create a cyclic iterator from the ordered letters
    cyclic_iterator = itertools.cycle(ordered_letters)
    
    # Select the first 'count' elements from the iterator
    selected_letters = [next(cyclic_iterator) for _ in range(count)]
    
    return selected_letters

# Example usage:
print(cyclic_letters(start='B'))

['B', 'C', 'D', 'E', 'F', 'A']


In [5]:
def relabel_NxNxN_N_state(state, start='A'):
    # Split the string into individual elements.
    state_list = state.split(';')
    
    dim = int(np.sqrt(len(state_list) // 6))
    dim_2 = dim**2
    letters = cyclic_letters(start=start)
    if state[0] == 'N':
        for i, val in enumerate(state_list):
            # Extract the number part from the string like 'N33' -> 33.
            num = int(val[1:])
            # Replace based on the given criteria.
            if 0 <= num < dim_2:
                state_list[i] = letters[0]
            elif dim_2 <= num < 2*dim_2:
                state_list[i] = letters[1]
            elif 2*dim_2 <= num < 3*dim_2:
                state_list[i] = letters[2]
            elif 3*dim_2 <= num < 4*dim_2:
                state_list[i] = letters[3]
            elif 4*dim_2 <= num < 5*dim_2:
                state_list[i] = letters[4]
            elif 5*dim_2 <= num < 6*dim_2:
                state_list[i] = letters[5]
    
    return ';'.join(state_list)  

In [6]:
def apply_moves(initial_state, moves, mmoves):
    state = initial_state.split(";")
    for move_name in mmoves.split('.'):
        state = [state[i] for i in moves[move_name]]
    
    return ';'.join(state)



In [7]:
def sym_rotations(dim, mmoves, new_state, sol_state, num_wildcards, wildcard=False):
    I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
    manipulations = (
            [''] + 
            I + 
            [i1 + '.' + i2 for i1 in I for i2 in I] +
            [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I] +
            [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]
    )
    
    for rotation in manipulations:
        temp_state = new_state
        if len(rotation) > 0:
            for move in rotation.split('.'):
                temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
        
        if wildcard: 
            total_correct = np.sum([sol_state[i] == ns for i, ns in enumerate(temp_state.split(';'))])
            if total_correct >= 6*(dim**2) - num_wildcards:
                print(f'solved id: {id}')
                if len(rotation) > 0:
                    mmoves += '.' + rotation
                break
        else:
            if temp_state == sol_state:
                print(f'solved id: {id}')
                if len(rotation) > 0:
                    mmoves += '.' + rotation
                break
                
    return temp_state, mmoves

In [8]:
def move_translation(dim):
    M = {}
    M["U"] = f'-d{dim-1}'
    M["R"] = "r0"
    M["B"] = f"-f{dim-1}"
    M["F"] = "f0"
    M["L"] = f"-r{dim-1}"
    M["D"] = "d0"
    
    
    I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'f', 'd']]
    for i, j in enumerate(['x', 'y', 'z']):
        M[j] = I[i]

    if dim > 3:
        M["Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["Rw"] = f"r0.r1"
        M["Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["Fw"] = f"f0.f1"
        M["Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["Dw"] = f"d0.d1"
        
    if dim >= 6:
        M["2Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["2Rw"] = f"r0.r1"
        M["2Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["2Fw"] = f"f0.f1"
        M["2Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["2Dw"] = f"d0.d1"

        width_max = dim // 2
        for i in range(3, width_max + 1):
            M[f"{i}Uw"] = f'-d{dim-i}.' + M[f"{i-1}Uw"]
            M[f"{i}Rw"] = M[f"{i-1}Rw"] + f'.r{i-1}'
            M[f"{i}Bw"] = f'-f{dim-i}.' + M[f"{i-1}Bw"]
            M[f"{i}Fw"] = M[f"{i-1}Fw"] + f'.f{i-1}'
            M[f"{i}Lw"] = f'-r{dim-i}.' + M[f"{i-1}Lw"]
            M[f"{i}Dw"] = M[f"{i-1}Dw"] + f'.d{i-1}'


    for m in list(M):
        M[m+"2"] = M[m] + '.' + M[m]
        if "-" in M[m]:
            M[m+"'"] = M[m].replace("-","")
        else:
            M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])
    
    return M

In [9]:
def get_moves(allowed_moves):
    moves = eval(allowed_moves)
    for move in list(moves):
        moves['-'+move] = np.argsort(moves[move]).tolist()
    return moves

def flip_diag_faces(M):
    # flip_diag_faces = "x' y Rw U Rw' U' F F U Rw U' Rw' F" # F"
    flip_diag_faces = "Rw U2 Rw' U2 Rw' F Rw2 U2 Rw' U2 Rw U Rw' F'"
    mm = '.'.join([M[m] for m in flip_diag_faces.split(' ')])
    return mm


def swap_faces(M):
    moves = "U F2 D Rw U2 Rw' U2 Rw' F Rw2 U2 Rw' U2 Rw U Rw' F' D' F2 U'"
    mm = '.'.join([M[m] for m in moves.split(' ')])
    return mm


def rotate_90(matrix):
    # Zip combines the elements from each of the sublists
    # The * operator unpacks the original list
    # The reversed() function is used to rotate it clockwise
    return [list(reversed(col)) for col in zip(*matrix)]
        

In [10]:
def solve(initial_state, solution_state, num_wildcards, moves, verbose=False):
    start='A'
    init_state = relabel_NxNxN_N_state(initial_state, start=start)
    sol_state = relabel_NxNxN_N_state(solution_state, start=start)
        
    # new_moves = random.choices(list(moves.keys()), k=1)
    
    # init_state = apply_moves(init_state, moves, '.'.join(new_moves))
    # print(init_state)
    state = state2ubl(init_state)
    print(f'Starting {id}')
    output = !./rubiks-cube-solver.py --state $state
    # return output
    sol = None
    if output[-1][:9] == 'Solution:':
        # print(output[-1])
        sol = output[-1].split(': ')[1]
    else:
        for n in range(1, 21):
            if 'Solution:' in output[-n]:
                sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                break
    if sol is None:
        print(output[-1])
        return output
                
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    # # new_moves_str = '.'.join(new_moves)
    # 
    # new_state = init_state
    # 
    # original_init = row['initial_state']
    # original_sol = row['solution_state'].split(';')
    # 
    # if num_wildcards >= 2:
    #     for idx, move in enumerate(mmoves.split('.')):
    #         new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
    #         original_init = ';'.join(list(np.asarray(original_init.split(';'))[np.array(moves[move])]))
    #         original_init, mmoves = sym_rotations(dim, mmoves, original_init, original_sol, num_wildcards)
    #         total_correct = np.sum([original_sol[i] == ns for i, ns in enumerate(original_init.split(';'))])
    #         if total_correct >= 6*dim**2 - num_wildcards:
    #             print('WILDCARD SOLUTION FOUND')
    #             mmoves = '.'.join(mmoves.split('.')[:idx+1])
    #             break
    #         
    # check rotations
    
    new_state = apply_moves(initial_state, moves, mmoves)
    temp_state, mmoves = sym_rotations(dim, mmoves, new_state, solution_state.split(';'), num_wildcards, wildcard=False)

    return mmoves, sol, output

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [11]:
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

for id in range(206, 207):
    row = puzzles.loc[id]
    if row['puzzle_type'][:4] == 'cube' and row['solution_state'][0] == 'N':
        dim = int(row['puzzle_type'].split('/')[-1])
        moves = get_moves(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        M = move_translation(dim)
        
        s = row['initial_state'].split(';')
        # staging_moves = "Bw2 Uw2"
        # staging_moves = "Bw2 Uw2" + " " + "B2 Dw R Bw2 D' Fw' U Lw' R' Dw2 Rw2 U L'" + " Dw2 Uw' Fw2 Bw2 Dw" + " Rw2 U Dw2 F D Fw2 Rw2 B' L2 D B2 D' L2 Rw2 U' Fw2"
        # 
        # staging_moves = '.'.join([M[i] for i in staging_moves.split()])
        
        staging_moves = \
            ("-f2.-f3.-f2.-f3.-d2.-d3.-d2.-d3.-f3.-f3.d0.d1.r0.-f2.-f3.-f2.-f3.-d0.-f0.-f1.-d3.r2.r3.-r0.d0.d1.d0"
             ".d1.r0.r1.r0.r1.-d3.r3.d0.d1.d0.d1.d2.d3.f0.f1.f0.f1.-f2.-f3.-f2.-f3.d0.d1.r0.r1.r0.r1.-d3.d0.d1.d0"
             ".d1.f0.d0.f0.f1.f0.f1.r0.r1.r0.r1.f3.-r3.-r3.d0.-f3.-f3.-d0.-r3.-r3.r0.r1.r0.r1.d3.f0.f1.f0.f1"
             ".-d3.f3.f3.r3"
             ".f0.f0.r0"
             ".-r3"
             ".d1.d1.r0.r0.f1.f1.r3.d2.d2.r3.d2.d2.r0.r0.f2.f2"
             # ".r3.r3"
             ".-r3"
             ".d1.d1.r0.r0.f1.f1.r3.d2.d2.r3.d2.d2.r0.r0.f2.f2"
             ".r3.r3"
             # # ".-d0"
             # ".f1.f1.d3.d3.r2.r2.-d0.f2.f2.-d0.f2.f2.d3.d3.r2.r2"
             # # ".-d0.-d0"
             # ".-f0.-f0"
             ".d1.d1.f0.f0.r2.r2.f3.d2.d2.f3.d2.d2.f3.f3.r2.r2"
             ".-f0"
             ".d1.d1.f0.f0.r2.r2.f3.d2.d2.f3.d2.d2.f3.f3.r2.r2"
             ".-f0"
             ".-d0"
             # ".f1.f1.d0.d0.r1.r1.d3.f2.f2.d3.f2.f2.d0.d0.r1.r1" # switch faces
             # ".f1.f1.d0.d0.r1.r1.d3.f2.f2.d3.f2.f2.d0.d0.r1.r1" # switch faces
             ".f1.f1.d0.d0.r1.r1.d3.f2.f2.d3.f2.f2.d0.d0.r2.r2" # no switch faces
             ".-d3.d0"
             ".f1.f1.d0.d0.r1.r1.d3.f2.f2.d3.f2.f2.d0.d0.r2.r2" # no switch faces
             ".d0.d0"
             )
        
        for idx, move in enumerate(staging_moves.split('.')):
            s = list(np.asarray(s)[np.array(moves[move])])
        mmoves, sol, output = solve(';'.join(s), row['solution_state'], row['num_wildcards'], moves) 

        # mmoves, sol, output = solve(row['initial_state'], row['solution_state'], row['num_wildcards'], moves, verbose=False) 
        # validation
        state = s #row['initial_state'].split(";")
        # mmoves = f'{new_moves_str}.{mmoves}'
        for move_name in mmoves.split('.'):
            state = [state[i] for i in moves[move_name]]
        
        
        while True:
            # state = apply_moves(row['initial_state'], moves, mmoves)
            print(state)
            try:
                assert row['solution_state'].split(";") == state
                print('SOLVED"')
                # mmoves_length = len(mmoves.split('.'))
                
                select_query = "SELECT count FROM solutions WHERE id = ?"
    
                # Execute the query
                cursor.execute(select_query, (id,))
                response = cursor.fetchone()
                best_moves_length = response[0]
                
                # if mmoves_length < best_moves_length:
                #     # Insert the moves into the database
                #     insert_query = ("INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)")
                #     cursor.execute(insert_query, (id, mmoves, mmoves_length, solution_method))
                #     conn.commit()
                break
            except AssertionError:
                print(f"assertion error for {id}")
                # print(state)
                break
        
                

# Commit the changes and close the connection
conn.commit()
conn.close()


Starting 206
['N0', 'N1', 'N2', 'N3', 'N4', 'N9', 'N6', 'N7', 'N8', 'N10', 'N5', 'N11', 'N12', 'N13', 'N14', 'N15', 'N16', 'N17', 'N18', 'N19', 'N20', 'N25', 'N26', 'N23', 'N24', 'N21', 'N22', 'N27', 'N28', 'N29', 'N30', 'N31', 'N32', 'N33', 'N34', 'N35', 'N36', 'N42', 'N38', 'N39', 'N40', 'N41', 'N37', 'N43', 'N44', 'N45', 'N46', 'N47', 'N48', 'N49', 'N50', 'N51', 'N52', 'N53', 'N57', 'N55', 'N56', 'N54', 'N58', 'N59', 'N60', 'N61', 'N62', 'N63', 'N64', 'N65', 'N66', 'N67', 'N68', 'N70', 'N69', 'N71', 'N72', 'N74', 'N73', 'N75', 'N76', 'N77', 'N78', 'N79', 'N80', 'N81', 'N82', 'N83', 'N84', 'N86', 'N89', 'N87', 'N88', 'N85', 'N90', 'N91', 'N92', 'N93', 'N94', 'N95']
assertion error for 206


In [12]:
phase1_sol = None
phase2_sol = None
phase34_sols = []

for line in output:
    if 'phase34_solution' in line:
        sol34 = eval(line.split('phase34_solution: ')[-1])
        sol3 = '.'.join([M[m] for m in sol34[0]])
        sol4 = '.'.join([M[m] for m in sol34[1]])
        phase34_sols.append(f'{sol3}.{sol4}')
        # break
    elif 'phase 1 solutions: ' in line:
        sol1 = eval(line.split('phase 1 solutions: ')[-1])

    elif 'phase 2 solutions: ' in line:
        sol2 = eval(line.split('phase 2 solutions: ')[-1])
        # sol3 = '.'.join([M[m] for m in sol34[0]])
        # sol4 = '.'.join([M[m] for m in sol34[1]])
        # phase34_sols.append(f'{sol3}.{sol4}')
        # break
    elif 'Solution before phase 2: ' in line:
        phase1_sol = eval(line.split('Solution before phase 2: ')[-1])
        phase1_sol = [m for m in phase1_sol if not m.startswith('COMMENT')]
    elif 'Solution before phase 3: ' in line:
        print(line)
        phase2_sol = eval(line.split('Solution before phase 3: ')[-1])
        phase2_sol = [m for m in phase2_sol if not m.startswith('COMMENT')]

# print(phase1_sol)
# print(phase2_sol)
# # line[0]
# print(sol34[0])
phase2_sol

Solution before phase 3: ["D'", 'B2', "R'", 'COMMENT_centers_staged,_edges_EOed_into_high/low_groups_(3_steps)']


["D'", 'B2', "R'"]

In [13]:
# mm = '.'.join([M[m] for m in phase1_sol])
# mm = '.'.join([M[m] for m in phase2_sol])
mm = ''
# for idx, move in enumerate(staging_moves.split('.')):
#     s = list(np.asarray(s)[np.array(moves[move])])

for i, sol34 in enumerate([0]):
    if mm:
        mm_att = f'{mm}.{sol34}'
    else:
        mm_att = sol34
    
    # mm_att = mm
    # temp = row['initial_state'].split(';').copy()
    temp = s.copy()
    dim_2 = 4**2
    
    # print(state[dim_2 * face + 5:dim_2 * face + 7])
    # print(state[dim_2 * face + 9:dim_2 * face + 11])
    # for idx, move in enumerate(mm_att.split('.')):
    #     temp = list(np.asarray(temp)[np.array(moves[move])])
    
    correct_faces = 0
    for face in range(6):
        center = [
            temp[dim_2 * face + 5:dim_2 * face + 7],
            temp[dim_2 * face + 9:dim_2 * face + 11]
        ]

        center_ideal = [[f'N{idx}' for idx in range(dim_2 * face + 5, dim_2 * face + 7)],
                  [f'N{idx}' for idx in range(dim_2 * face + 9, dim_2 * face + 11)]]

        print(center)
        # print()
        print(center_ideal)
        print()
        # break
        for _ in range(4):
            center = rotate_90(center)
            if center == center_ideal:
                correct_faces += 1
                break
                
    if correct_faces == 6:
        break
    if correct_faces > 2:
        print(correct_faces)
    break

[['N5', 'N6'], ['N9', 'N10']]
[['N5', 'N6'], ['N9', 'N10']]

[['N21', 'N22'], ['N25', 'N26']]
[['N21', 'N22'], ['N25', 'N26']]

[['N37', 'N38'], ['N41', 'N42']]
[['N37', 'N38'], ['N41', 'N42']]

[['N53', 'N54'], ['N57', 'N58']]
[['N53', 'N54'], ['N57', 'N58']]

[['N69', 'N70'], ['N73', 'N74']]
[['N69', 'N70'], ['N73', 'N74']]

[['N85', 'N86'], ['N89', 'N90']]
[['N85', 'N86'], ['N89', 'N90']]


In [14]:
mm_att.split('.')
mm

AttributeError: 'int' object has no attribute 'split'

In [ ]:
' '.join(sol.split()[:17-1])

In [ ]:
s = row['initial_state'].split(';')
for idx, move in enumerate(M["Bw2'"].split('.')):
    s = list(np.asarray(s)[np.array(moves[move])])
    
s[90 - 1]

In [ ]:
s[86 - 1 ]

In [ ]:
staging_moves